Create new DB

Create new User - DHAN, Give UNLIMITED QUOTA, Enable Web Access, Give Connect and Resource Role

Export Apex app from Old
Export dump from old DB

In [ ]:
expdp userid=dhan/Welcome12345#@ssn1 FILE=DHAN_expdp_1402.dmp DIRECTORY=data_pump_dir

Put dump in bucket from OLD DB

In [ ]:
DECLARE
my_blob_data BLOB;
BEGIN 
DBMS_CLOUD.PUT_OBJECT(credential_name => 'SSN_CRED2',
object_uri => 'https://objectstorage.ap-hyderabad-1.oraclecloud.com/p/7lO-9rI8x4rDhIk9K6Oup50XN73A_0AeZzQpK8VEQl0CZh34aOsun6_2xqUM6ETT/n/axdpselyzomz/b/bucket-20231222-0929/o/SSNO.dmp',
     contents => my_blob_data); 
END;
 

Create Credential in new DB

In [ ]:
begin
  DBMS_CLOUD.create_credential(
    credential_name => 'SSN_CRED2',
    username        => 'sreejithsna@gmail.com',
    password        => 'Amos@123');
end;
/

Create Preauthenticated URL for Dump from GCS Bucket - bucket-SSN1

Provide this URL in the import command

Import Dump 

In [ ]:
DECLARE
    -- replace with your values
    exported_schema VARCHAR2(64)  := 'DHAN';
    import_schema   VARCHAR2(64)  := 'DHAN'; -- in case you want to remap schema
    data_pump_dir   VARCHAR2(64)  := 'DATA_PUMP_DIR';
    dump_file_name  VARCHAR2(256) := 'https://objectstorage.ap-mumbai-1.oraclecloud.com/p/0BcHReOzTpYdkWSMuqx9Upm--d1Aq7nNLHeYZMETjdMwTC7dXTy7aaAAxBIB8IH6/n/bmiyb53g43ha/b/bucket-SSN1/o/DHAN_expdp_1402.dmp';
    credential_name VARCHAR2(64)  := 'SSN_CRED1';
    parallel        NUMBER        := 4;
 
    job_handle      NUMBER;
    job_name        VARCHAR2(64);
    job_status      VARCHAR2(128);
    output_message  VARCHAR2(1024);
BEGIN
    job_name := dbms_scheduler.generate_job_name('import');
    job_handle := dbms_datapump.open(operation => 'IMPORT', job_mode => 'SCHEMA', job_name => job_name); 
    dbms_datapump.add_file(handle => job_handle, filename => dump_file_name, directory => credential_name, filetype => dbms_datapump.ku$_file_type_uridump_file); 
    dbms_datapump.add_file(handle => job_handle, filename => import_schema || '_import1.log', directory => data_pump_dir, filetype => 3);
    dbms_datapump.metadata_remap(job_handle, 'REMAP_SCHEMA', exported_schema, import_schema);
    dbms_datapump.metadata_filter(handle => job_handle, name => 'SCHEMA_EXPR', value => 'IN(''' || exported_schema || ''')');
    dbms_datapump.set_parallel(handle => job_handle, degree => parallel);
    dbms_datapump.start_job(handle => job_handle, skip_current => 0, abort_step => 0); 
    dbms_datapump.wait_for_job(handle => job_handle, job_state => job_status);
    output_message := 'Data Pump Import Execution: ''' || job_status || '''';
    dbms_output.put_line(output_message);
END;
/

Configure Apex pointing to DHAN
Create Workspace
Import Apex Application to new Workspace 

Update TNS in OCI Instance with new connect description
Download Wallet
Copy .sso file to OCI Instance /home/opc/instantclient_19_15/network/admin
    scp -i ssh-key-2022-06-22.key  Wallet_SSN2/cwallet.sso opc@140.238.255.42:/home/opc/instantclient_19_15/network/admin

Start Application


In [ ]:
DECLARE
TYPE ty_object IS TABLE OF VARCHAR2(4000) INDEX BY BINARY_INTEGER; 
tab_object ty_object;

BEGIN

     
                        SELECT 'drop ' || object_type || ' ' ||owner||'."' || object_name ||'"' ||DECODE( object_type,'TABLE',
                                ' CASCADE CONSTRAINTS PURGE',NULL) BULK COLLECT
                        INTO tab_object
                        FROM all_objects
                        WHERE object_type NOT IN ('INDEX','JAVA CLASS','TYPE','TABLE PARTITION', 'LOB', 'PACKAGE BODY')
                                AND owner NOT  IN ( 'SYS','SYSTEM')
                                --AND 1=2
                                AND owner = 'DHAN';

                        FOR i IN 1 .. tab_object.COUNT
                        LOOP
                                BEGIN
                                    --DBMS_OUTPUT.PUT_LINE( tab_object(i));
                                        EXECUTE IMMEDIATE tab_object(i);

                                EXCEPTION

                                WHEN OTHERS THEN
                                        NULL;

                                END;

                        END LOOP;
                        tab_object.delete;

                END;

END;
/